Import modules

In [1]:
import boto3
import json
import os
from dotenv import load_dotenv 
import cv2
import ast

Get environment variables and create AWS Session

In [2]:

dotenv_path = os.path.join("..",'.env')
load_dotenv(dotenv_path)

#Connect to AWS Ressource, Credentials are optionals as we are accessing from the same AWS user
client=boto3.client('rekognition',aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("REGION_NAME"))


#Define bucket name to access

#Folder of real classroom photos
classroomFolder=os.path.join("..","ImagesRekognition","RealClassroomPhotos")

Take all images of the folder and for each draw detected faces and save it to FacesDetectedInClassroom

In [3]:

#Cycle that only draws rectangles where there are faces according to Rekognition's algorithm
i=1
boundingBoxDetectedFaces=[]

for file in os.listdir(classroomFolder):
    camera_local_file = os.path.join("..","ImagesRekognition","RealClassroomPhotos",file)
    #convert files to bytes in order to be able to read it

    with open(camera_local_file, 'rb') as image_file:
        camera_file_bytes = image_file.read()
    responseDetection=client.detect_faces( Image={'Bytes': camera_file_bytes})
    # Read the image with OpenCV
    image = cv2.imread(camera_local_file)
    image_height, image_width, _ = image.shape

    boundingBoxDetectedFacesithImage=[]

    # Draw bounding boxes for each detected face
    for face in responseDetection['FaceDetails']:
        bbox = face['BoundingBox']
        left = int(bbox['Left'] * image_width)
        top = int(bbox['Top'] * image_height)
        width = int(bbox['Width'] * image_width)
        height = int(bbox['Height'] * image_height)
        cv2.rectangle(
            image, 
            (left, top), 
            (left + width, top + height), 
            (37, 173, 250), 
            2  
        )
        boundingBoxDetectedFacesithImage.append((left, top, width,height))
    #Each array element represents a sub-array of tuples containing the bonding boxes    
    boundingBoxDetectedFaces.append(boundingBoxDetectedFacesithImage) 
    # Print number of faces detected
    print(f"{len(responseDetection['FaceDetails'])} Faces were detected")

    # Save the annotated image
    output_path =os.path.join("..","ImagesRekognition","FacesDetectedInClassroom",f"faces_detected_{i}.jpeg")
    cv2.imwrite(output_path, image)
    #Increase the photo's index
    i=i+1


21 Faces were detected
21 Faces were detected
9 Faces were detected
9 Faces were detected
9 Faces were detected
18 Faces were detected
19 Faces were detected
9 Faces were detected
9 Faces were detected
9 Faces were detected
9 Faces were detected
9 Faces were detected
9 Faces were detected
9 Faces were detected
18 Faces were detected
16 Faces were detected
17 Faces were detected
17 Faces were detected
15 Faces were detected
17 Faces were detected
17 Faces were detected
17 Faces were detected
16 Faces were detected
19 Faces were detected
18 Faces were detected
15 Faces were detected
17 Faces were detected
17 Faces were detected
15 Faces were detected
18 Faces were detected


In [29]:
for file in os.listdir(classroomFolder):
    if (int(str(file).split(".")[0][-1])>3):
        print(str(file).split(".")[0][-1]) 

In [ ]:

def calculate_iou_detection(box1, box2):
    """
    Calculate IoU between 2 boxes.
    box1, box2: tuples like (x, y, width, height)
    return IoU value that ranges from 0 to 1
    """
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    x_intersect_1 = max(x1, x2)
    y_intersect_1 = max(y1, y2)
    x_intersect_2 = min(x1 + w1, x2 + w2)
    y_intersect_2 = min(y1 + h1, y2 + h2)
    
    if x_intersect_2 <= x_intersect_1 or y_intersect_2 <= y_intersect_1:
        return 0.0
    
    area_intersect = (x_intersect_2 - x_intersect_1) * (y_intersect_2 - y_intersect_1)
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    
    area_union = area_box1 + area_box2 - area_intersect
    
    iou = area_intersect / area_union
    
    return iou




In [3]:
true=True


with open(os.path.join("..","Data","groundTruth.txt"), "r") as file:
    content = file.read()


groundTruth=ast.literal_eval(content)

m=1
#The N-Element contains tuples of the N-Th image with informations of students to be recognized
tuples=[]
for element in groundTruth:
    i=1
    #(f"photo {m}-----------------------------------------")
    filteredgroundTruthObject=element[next(iter(element))]["regions"]
    tuplesnthPhoto=[]
    for element in filteredgroundTruthObject:
        #print(f"Person {i}-----")
        region=element["shape_attributes"]
        tuplesnthPhoto.append((region["x"],region["y"], region["width"], region["height"]))
        i=i+1
    tuples.append(tuplesnthPhoto)
    m=m+1

In [ ]:
i=0
TPTotal=0
TNTotal=0
FPTotal=0
FNTotal=0

for element in tuples:
    TP=0
    TN=0
    FN=0
    FP=0
    verifiedPredictions=0    
    print(f"------Results for image {i+1}------")
    for face in element: #Compare each face of ground truth with the predictions' array. If found, add TP, if not identified, add FN
        found=False
        for detectedFace in boundingBoxDetectedFaces[i]:
            IoU=calculate_iou_detection(face, detectedFace)
            if (IoU>=0.5):
                found=True
                TP=TP+1
                verifiedPredictions=verifiedPredictions+1
                break
        if (found==False):
            FN=FN+1
    FP=FP+(len(boundingBoxDetectedFaces[i])-verifiedPredictions)
    print(F"TP: {TP}, TN:{TN}, FP:{FP} FN:{FN}")
    TPTotal=TPTotal+TP
    TNTotal=TNTotal+TN
    FPTotal=FPTotal+FP
    FNTotal=FNTotal+FN


    i=i+1


print("----Total----")
print(f"TP: {TPTotal}, TN: {TNTotal}, FP:{FPTotal},FN: {FNTotal}")      



Take all images of the folder and all student's personal image and draw a unique color rectangle if recognized

In [19]:
colors = [
    (255, 0, 0),     
    (0, 255, 0),     
    (0, 0, 255),     
    (255, 255, 0),   
    (255, 0, 255),   
    (0, 255, 255),  
    (128, 0, 0),     
    (0, 128, 0),     
    (0, 0, 128),      
    (128, 128, 0),    
    (128, 0, 128),    
    (0, 128, 128),    
    (64, 0, 128),     
    (128, 128, 255),  
    (0, 255, 128),   
    (128, 255, 128),  
    (192, 64, 64),   
    (64, 192, 192),  
    (255, 128, 192), 
    (32, 64, 255)    
]
#Array that contains all faces recognized in all test images. Each photo corresponds to the n-th array element
boundingBoxRecognizedFaces=[]
recognized_counter_array=[]
m=1
photo_index=1

#Create image of the class with all faces recognized in it
for file in os.listdir(classroomFolder):
    camera_local_file = os.path.join("..","ImagesRekognition","RealClassroomPhotos", file)
    print(f"Using index {photo_index}")

    if (photo_index>=1 and photo_index<=2) or  (photo_index>=23 and photo_index<=30):
        selectedCourse="StudentsRobotics"
        student_array=["AlbaQuiroz", "AlejandroCasallas","AlejandroVelasco","AlexanderDominguez","AndresAnillo","CarlosBornacelly","DavidChaparro"
                    ,"EstebanToro","GabrielCastanez","GiohanOlivares","JesusCotes","JoseVilla","JuanMolina" ,"ManuelRangel","MariaBerdugo","MariaLopez",
                    "MauricioDeLaHoz", "SergioFonseca","YordiRochel"]    
    elif (photo_index>=3 and photo_index<=5) or (photo_index>=8 and photo_index<=14):
        selectedCourse="StudentsElectronicDesign1"
        student_array=["AllysonSalom", "AngieValencia","DanielDelgado","DiegoGomez", "DylanDeOro","FelipeOspino","HabidAbdala","JuanLozano","WilmanDaza"]

    elif (photo_index>=6 and photo_index<=7) or  (photo_index>=15 and photo_index<=22):
        selectedCourse="StudentsElectronicDesign2"
        student_array=["AliRada","AndreaQuintero","AndresFabregas","AndresNarvaez","BrayanRubiano","DiegoGomez","EdwinUtria","GabrielaBecerra",
                    "JuanHernandez","JuanQuintero","JuanSanchez","LucianaDeLaRosa","MariaFerrer","MayraCarreno",
                    "OctavioMorales","SamirBarcelo","SteevenBallena"]

    #convert files to bytes in order to be able to read it
    with open(camera_local_file, 'rb') as image_file:
        camera_file_bytes = image_file.read()
    responseDetection=client.detect_faces( Image={'Bytes': camera_file_bytes})
    image = cv2.imread(camera_local_file)

    k=0
    recognized_counter=0

    boundingBoxRecognizedFacesithImage=[]

    for student_name in student_array:
        student_found = False  # Flag to check if the student was found
        
        student_id_photo=os.path.join("..","ImagesRekognition",selectedCourse, f"{student_name}.jpg")
        if not os.path.exists(student_id_photo):
            student_id_photo=os.path.join("..","ImagesRekognition",selectedCourse, f"{student_name}.jpeg")
            
        with open(student_id_photo, 'rb') as image_file:
            student_id_photo_bytes = image_file.read()

        #Look for the face in SourceImage evaluating all the faces in TargetImage
        responseRecognition=client.compare_faces(
            SourceImage={'Bytes': student_id_photo_bytes},
            TargetImage={'Bytes': camera_file_bytes},
            SimilarityThreshold=95
        )
        # Read the target image with OpenCV

        # Get image dimensions
        image_height, image_width, _ = image.shape

        # Draw bounding boxes for matched faces
        for faceMatch in responseRecognition['FaceMatches']:
            # Get bounding box coordinates
            bbox = faceMatch['Face']['BoundingBox']
            
            # Convert normalized coordinates to pixel coordinates
            left = int(bbox['Left'] * image_width)
            top = int(bbox['Top'] * image_height)
            width = int(bbox['Width'] * image_width)
            height = int(bbox['Height'] * image_height)
        
            # Draw green rectangle (to differentiate from previous red detection boxes)
            cv2.rectangle(
                image, 
                (left, top), 
                (left + width, top + height), 
                colors[k],  # Green color in BGR
                2,
            )
            
            padding = 5  # Espacio entre el rectángulo y el cuadro de texto
            text_box_height = 25  # Altura del cuadro de texto
            # Agregar texto dentro del recuadro de texto
            cv2.putText(image, f"{student_name}", (left + 10, top - padding - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[k], 1)

            recognized_counter=recognized_counter+1
            print('Face at coordinates ' + str(bbox) + ' matches with ' + str(faceMatch['Similarity']) + '% confidence'+ f"and is likely {student_name}")

            #Save the identity and region as a tuple in the n-th array element that corresponds to the n-th image
            boundingBoxRecognizedFacesithImage.append((student_name,left,top,width,height))
            student_found = True 
        
        if not student_found:
            boundingBoxRecognizedFacesithImage.append((student_name, 0, 0, 0, 0))
            print(f"Student {student_name} not found in the image")
            
        k=k-1
    boundingBoxRecognizedFaces.append(boundingBoxRecognizedFacesithImage)
    recognized_counter_array.append(recognized_counter)
    # Save the annotated image
    output_path = os.path.join("..","ImagesRekognition","StudentsFoundInClassroom95", f"faces_in_classroom_{m}_{recognized_counter}.jpeg")
    cv2.imwrite(output_path, image)
    print(f"Annotated image saved to {output_path}")
    m=m+1
    photo_index=photo_index+1

Using index 1
Student AlbaQuiroz not found in the image
Face at coordinates {'Width': 0.02335977554321289, 'Height': 0.039030011743307114, 'Left': 0.5729531049728394, 'Top': 0.41381317377090454} matches with 99.61832427978516% confidenceand is likely AlejandroCasallas
Face at coordinates {'Width': 0.029462289065122604, 'Height': 0.05213397368788719, 'Left': 0.269317626953125, 'Top': 0.49795401096343994} matches with 99.83786010742188% confidenceand is likely AlejandroVelasco
Face at coordinates {'Width': 0.016162682324647903, 'Height': 0.027424240484833717, 'Left': 0.4776175022125244, 'Top': 0.3491821885108948} matches with 95.22370147705078% confidenceand is likely AlexanderDominguez
Student AndresAnillo not found in the image
Face at coordinates {'Width': 0.04340658336877823, 'Height': 0.07031898200511932, 'Left': 0.4438689649105072, 'Top': 0.5599902868270874} matches with 99.98633575439453% confidenceand is likely CarlosBornacelly
Face at coordinates {'Width': 0.048601578921079636, 

Obtain tuples from base line (Manual correction and verification)

In [9]:

with open(os.path.join("..","Data","faceGallery.txt"), "r") as file:
    content = file.read()

faceGallery=ast.literal_eval(content)


m=1
#The N-Element contains tuples of the N-Th image with informations of students to be recognized
tuples=[]
for element in faceGallery:
    i=1
    #(f"photo {m}-----------------------------------------")
    filteredFaceGalleryObject=element[next(iter(element))]["regions"]
    tuplesnthPhoto=[]
    for element in filteredFaceGalleryObject:
        #print(f"Person {i}-----")
        region=element["shape_attributes"]
        identity=element["region_attributes"]
        #print(f"x = {region["x"]} \ny = {region["y"]} \nwidth = {region["width"]}\nheight = {region["height"]} \nidentifier = {identity["name"]}")
        tuplesnthPhoto.append((identity["name"],region["x"],region["y"], region["width"], region["height"]))
        i=i+1
    tuples.append(tuplesnthPhoto)
    m=m+1


Code to calculate IoU

In [10]:

def calculate_iou(box1, box2):
    """
    Calculate IoU between 2 boxes.
    box1, box2: tuples like (x, y, width, height)
    return IoU value that ranges from 0 to 1
    """
    name1,x1, y1, w1, h1 = box1
    name2, x2, y2, w2, h2 = box2
    
    x_intersect_1 = max(x1, x2)
    y_intersect_1 = max(y1, y2)
    x_intersect_2 = min(x1 + w1, x2 + w2)
    y_intersect_2 = min(y1 + h1, y2 + h2)
    
    if x_intersect_2 <= x_intersect_1 or y_intersect_2 <= y_intersect_1:
        return 0.0
    
    area_intersect = (x_intersect_2 - x_intersect_1) * (y_intersect_2 - y_intersect_1)
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    
    area_union = area_box1 + area_box2 - area_intersect
    
    iou = area_intersect / area_union
    
    return iou


Iterate through the base line and compare it to the results obtained by Rekognition. Then calculate TP, TN, FP, FN

In [20]:
#It searchs based on face gallery/Base Line: It compares my definition of the existance of specific students and compares it to the found studens
#The result wouldn't be affected if the search direction is reversed

i=0
TPTotal=0
TNTotal=0
FPTotal=0
FNTotal=0
#Loop through all the photos
for photoObject in tuples:
    print(f"Results for photo {i+1}")
    TP=0
    TN=0
    FP=0
    FN=0
    #Obtain the array of faces recognized with Rekognition in that photo
    tuplesToCompare=boundingBoxRecognizedFaces[i]

    listOfNamesBaseline=[]
    for element in photoObject:
        listOfNamesBaseline.append(element[0])

    listOfNamesModel=[]
    for element in tuplesToCompare:
        listOfNamesModel.append(element[0])


    for element in photoObject:
        if(element[0] in listOfNamesModel):
            for identity in tuplesToCompare:
                IoU=calculate_iou(element,identity)

                #Case True Positive: Identity and bounding box match
                if(IoU>=0.5 and element[0]==identity[0]):
                    TP=TP+1

                #Case False Positive: no identity match, however, the bounding boxes are similar
                if(IoU>=0.5 and element[0]!=identity[0]):
                    FP=FP+1
                    print(f"conflict between {element[0]} and {identity[0]} in element {element}")

                #Case True Negative: No bounding box match. However, the identity is different
                if(IoU<0.5 and element[0]!=identity[0]):
                    TN=TN+1
                #Case False Negative: The identity matches. However, bonding boxes do not correspond
                if(IoU<0.5 and element[0]==identity[0]):
                    FN=FN+1 
            
            #Correction to take into account if the model doesn't mistake identities of strangers: Detected Faces-Faces identified        
            TN=TN+len(boundingBoxDetectedFaces[i])-len(listOfNamesModel)
        else:
            FN=FN+1
            TN=TN+len(boundingBoxDetectedFaces[i])-1
            print(f"{element[0]} not in model's list and adds 1 FN and {len(boundingBoxDetectedFaces)-1} TN")
            
    TPTotal=TPTotal+TP
    TNTotal=TNTotal+TN
    FPTotal=FPTotal+FP
    FNTotal=FNTotal+FN
  
    #print(recognized_counter_array[i])
    print(f"TP : {TP} \nTN : {TN} \nFP : {FP} \nFN : {FN}")
    i=i+1

print("----Total----")
print(f"TP: {TPTotal}, TN: {TNTotal}, FP:{FPTotal},FN: {FNTotal}") 

Results for photo 1
TP : 11 
TN : 360 
FP : 0 
FN : 7
Results for photo 2
TP : 8 
TN : 360 
FP : 0 
FN : 10
Results for photo 3
TP : 6 
TN : 56 
FP : 0 
FN : 1
Results for photo 4
TP : 6 
TN : 56 
FP : 0 
FN : 1
Results for photo 5
TP : 7 
TN : 56 
FP : 0 
FN : 0
Results for photo 6
TP : 12 
TN : 289 
FP : 0 
FN : 5
Results for photo 7
TP : 12 
TN : 306 
FP : 0 
FN : 5
Results for photo 8
TP : 5 
TN : 64 
FP : 0 
FN : 3
Results for photo 9
TP : 5 
TN : 56 
FP : 0 
FN : 2
Results for photo 10
TP : 6 
TN : 64 
FP : 0 
FN : 2
Results for photo 11
TP : 5 
TN : 64 
FP : 0 
FN : 3
Results for photo 12
TP : 5 
TN : 56 
FP : 0 
FN : 2
Results for photo 13
TP : 6 
TN : 56 
FP : 0 
FN : 1
Results for photo 14
TP : 4 
TN : 56 
FP : 0 
FN : 3
Results for photo 15
TP : 4 
TN : 170 
FP : 0 
FN : 6
Results for photo 16
TP : 4 
TN : 150 
FP : 0 
FN : 6
Results for photo 17
TP : 2 
TN : 96 
FP : 0 
FN : 4
Results for photo 18
TP : 4 
TN : 128 
FP : 0 
FN : 4
Results for photo 19
TP : 3 
TN : 98 
FP : 0